# Basic Usage of EPyT

### Outline
* Initialize EPANET-Python Toolkit
* Running your first analysis
* Other method to run EPANET
* Plot results
* Iteratively (step-by-step) hydraulic analysis
* Iteratively (step-by-step) quality analysis

### Initialize EPANET Python Toolkit (EPyT)

You should always begin with this command to import the toolkit.

In [ ]:
%pip install epyt

In [ ]:
from epyt import epanet

### Load a benchmark network
Decide which benchmark network to use and load the network

In [ ]:
# Decide which network to load from the "/networks/" folder  
filename = 'net2-cl2.inp' # You can also try 'net2-cl2.inp', 'Net3.inp', etc.

# Call epanet class and load all data and functions in G structure
G = epanet(filename)
G.plot()

### Running your first analysis
There are 4 ways to run a simulation using the Toolkit:

1. &nbsp;&nbsp;&nbsp;&nbsp;Using the batch hydraulic/quality analysis function
2. &nbsp;&nbsp;&nbsp;&nbsp;Using the iterative (step-by-step) hydraulic/quality analysis function
3. &nbsp;&nbsp;&nbsp;&nbsp;Using the ENepanet function binary file
4. &nbsp;&nbsp;&nbsp;&nbsp;Using the command line and the binary file



In [ ]:
# Set simulation duration at hours 
hrs = 48 
G.setTimeSimulationDuration(3600*hrs)

### Execute the simulation using the batch hydraulic analysis method


In [ ]:
H = G.getComputedHydraulicTimeSeries()
G.getAllAttributes(H)

### Execute the simulation using the batch quality analysis method

In [ ]:
Q = G.getComputedQualityTimeSeries()
G.getAllAttributes(Q)

### Other methods  to run EPANET

#### Method using the EPANET executable

In [ ]:
Hexe = G.getComputedTimeSeries()
G.getAllAttributes(Hexe)

### Method using the ENepanet library function

In [ ]:
Hbatch = G.getComputedTimeSeries_ENepanet()
G.getAllAttributes(Hbatch)

### Plot results

In [ ]:
import datetime
import matplotlib.pyplot as plt
import numpy as np

# Get pressure in meters
P = H.Pressure
G.printv(P)
F = H.Flow
G.printv(F)
C = Q.NodeQuality
G.printv(C)

In [ ]:
T = H.Time.astype(float)  
Tcl = Q.Time.astype(float)  
TD = [datetime.datetime(2024, 7, 1, 0, 0) + datetime.timedelta(seconds=int(t)) for t in T]  
TQ = [datetime.datetime(2024, 7, 1, 0, 0) + datetime.timedelta(seconds=int(t)) for t in Tcl] 

plt.rc('xtick', labelsize=5)
plt.rc('ytick', labelsize=7)
fig, ax = plt.subplots(figsize=(4, 3))

fig.set_figwidth(8)
ax.plot(TD, P[:, 5])
ax.grid(True)
ax.set_title(f"Pressure in Node {G.getNodeNameID(6)}", fontsize=8)
ax.set_ylabel("Pressure (meters)", fontsize=7)
ax.set_xlabel("Time", fontsize=7)

pipeindex = G.getLinkIndex('1')
fig, ax = plt.subplots(figsize=(4, 3))
fig.set_figwidth(8)
ax.plot(TD, F[:, pipeindex])
ax.grid(True)
ax.set_title(f"Flow in Pipe {G.getLinkNameID(pipeindex)}", fontsize=8)
ax.set_ylabel("Flow (CMH)", fontsize=7)
ax.set_xlabel("Time", fontsize=7)

fig, ax = plt.subplots(figsize=(4, 3))
fig.set_figwidth(8)
ax.plot(TQ, C[:, 5])
ax.grid(True)
ax.set_title(f"Chlorine concentration in Node {G.getNodeNameID(1)}", fontsize=8)
ax.set_ylabel("Free Chlorine (mg/L)", fontsize=7)
ax.set_xlabel("Time", fontsize=7)

plt.show()

# Your turn: Plot Chlorine in node with index 10 

![Alt text](images/pic1.jpeg)

### Plot node pressures for specific nodes 

In [ ]:
node_indices = [1,2,3] 
print("Node_indeces = ",node_indices)
node_names = [G.getNodeNameID(idx) for idx in node_indices]
Units = G.getUnits()
FlowUnits = G.getFlowUnits()
print("Flow_Units = ", FlowUnits)

fig, axs = plt.subplots(3, 1, figsize=(8, 6)) 

for k, node_index in enumerate(node_indices):
    ax = axs[k] 
    node_index_py = node_index - 1 # Python indexing starts from 0
    fig.set_figwidth(7)
    ax.plot(TD, P[:, node_index_py])
    ax.grid(True)
    ax.set_title(f'Pressure for the node id "{node_names[k]}"', fontsize=8)
    ax.set_ylabel(f'Pressure ({G.NodePressureUnits})', fontsize=7) 
    ax.set_xlabel('Time (hrs)', fontsize=7)

plt.tight_layout() # Automatically adjusts spacing to fit the subplots
plt.show()

![Alt text](images/pic2.jpeg)


### Iteratively (step-by-step) hydraulic analysis

In [ ]:
# 4 steps to run hydraulics

# Step 1
G.openHydraulicAnalysis()  

# Step 2
G.initializeHydraulicAnalysis() 

tstep = 1
P2 = []
T2 = []
D2 = []
H2 = []
F2 = []

while tstep > 0:
    
    # Step 3
    ttmp = G.runHydraulicAnalysis()  
    P2.append(G.getNodePressure())
    D2.append(G.getNodeActualDemand())
    H2.append(G.getNodeElevations())
    #H2.append(G.getNodeHydaulicHead())
    F2.append(G.getLinkFlows())
    T2.append(ttmp)
    
    # Step 4
    tstep = G.nextHydraulicAnalysisStep()  

G.closeHydraulicAnalysis()

In [ ]:
# Print pressures P2
print(P2)

#### Plot figures

In [ ]:
TD = [datetime.datetime(2024, 7, 1, 0, 0) + datetime.timedelta(seconds=int(t)) for t in T2]  

fig, ax = plt.subplots()
for i, node_index in enumerate(node_indices):
    node_index_py = node_index - 1
    ax.plot(TD, [row[node_index_py] for row in P2], label=f'Node {node_index}')
ax.set_title(f'Pressure for first 3 nodes ', fontsize=10)
ax.set_xlabel('Time (seconds)', fontsize=9)
ax.set_ylabel(f'Pressure({G.NodePressureUnits})', fontsize=8)
fig.set_figwidth(7)
ax.legend(loc='upper right', fontsize=7)
plt.show()

### Iteratively (step-by-step) quality analysis

In [ ]:
# Requires solveCompleteHydraulics - ignored because we run hydraulic analysis beforehand.

G.openQualityAnalysis()
G.initializeQualityAnalysis()
tleft = 1
QT = []
QsN = []
QsL = []

while tleft > 0:
    ttmp = G.runQualityAnalysis()
    QsN.append(G.getNodeActualQuality())
    QsL.append(G.getLinkActualQuality())
    QT.append(ttmp)
    tleft = G.stepQualityAnalysisTimeLeft()

G.closeQualityAnalysis()

#### Plot figures

In [ ]:
fig, ax = plt.subplots()
ax.plot(QT, [row[4] for row in QsN])  # Assuming the node index is 5 (Python indexing starts from 0)

ax.set_xlabel('Time', fontsize=7)
ax.set_ylabel('Node Actual Quality', fontsize=7)
ax.set_title('Node Actual Quality over Time', fontsize=8)

fig.set_figwidth(6)
plt.show()